In [1]:
import pandas as pd 

In [2]:
from datetime import date

In [3]:
import numpy as np

#### submittion

In [4]:
submittion = pd.read_csv('../data/raw/ejemploRespuesta.csv', header=None, names=['id_pos','unidades_pred'])

#### venta

In [5]:
ventas = pd.read_csv('../data/raw/ventas.txt', sep='|')

In [6]:
ventas.rename(columns={'fecha':'fecha_venta'}, inplace=True)

In [7]:
ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'])

In [8]:
canales = ventas[ventas.groupby(['id_pos'])['fecha_venta'].transform(max) == ventas['fecha_venta']][['id_pos', 'canal']]

In [9]:
canales.drop_duplicates(inplace=True)

In [10]:
canales.id_pos.nunique()

12302

In [11]:
canales.id_pos.count()

12304

In [12]:
canales_agg = canales.groupby('id_pos').size().reset_index()

In [13]:
canales_agg.columns = ['id_pos', 'numcases']

In [14]:
canales_agg[canales_agg.numcases > 1]

,id_pos,numcases
4887,8243,2
5299,8900,2


In [15]:
canales[canales.id_pos.isin(canales_agg[canales_agg.numcases > 1]['id_pos'].to_list())]

,id_pos,canal
236452,8243,ALMACEN
238396,8243,MAXIKIOSCO
265361,8900,MAXIKIOSCO
265461,8900,ALMACEN


In [16]:
canales['canal'] = np.where(canales.id_pos.isin(canales_agg[canales_agg.numcases > 1]['id_pos'].to_list()), 'ALMACEN', canales['canal'])

In [17]:
canales.drop_duplicates(inplace=True)

In [18]:
canales.id_pos.nunique()

12302

In [19]:
canales.id_pos.count()

12302

In [20]:
del ventas

#### pos

In [21]:
pos = pd.read_csv('../data/raw/pos.csv')

In [22]:
pos = pos[pos['id_pos'].isnull() != True]

In [23]:
pos = pos[pos['id_pos']!='Not Available']

In [24]:
# convertimos a int para poder cruzar con informacion de venta
pos['id_pos'] = pos['id_pos'].astype(int)

#### merging

In [25]:
submittion.shape

(12897, 2)

In [26]:
submittion = pd.merge(submittion, pos, how='inner', on='id_pos')

In [27]:
submittion.shape

(12897, 34)

In [28]:
submittion = pd.merge(submittion, canales, how='left', on='id_pos')

In [29]:
submittion.shape

(12897, 35)

In [30]:
submittion.to_csv('../data/raw/submission.csv', index=False, sep=';')